In [1]:
import numpy as np
import pandas as pd
import datetime
import pyreadr
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Coronanet dataset

In [ ]:
# 1) conversion des dates en datetime
# 2) date start devient la date annonced quand la date start est avant la date annonced
# 3) drop tous les duplicate sur la colonne policy_id 
# 4) création d'une nouvelle base de données (Lou)
# 5) -

In [2]:
# import data
df = pd.read_csv('../data/coronanet_release_allvars.csv', encoding='mac_roman')

In [5]:
def Tristans_columns(df):
    df['date_announced'] = pd.to_datetime(df['date_announced'])
    df['date_start'] = pd.to_datetime(df['date_start'])
    df['date_end'] = pd.to_datetime(df['date_end'])
    df['date_announced'].fillna(df['date_start'], inplace = True)
    df['date_start'].clip(lower=df['date_announced'], inplace=True)
    df_select = df[['policy_id', 'date_start', 'country', 
           'type', 'ISO_A3','compliance']]
    df_select.drop_duplicates(subset=['policy_id'], inplace=True)
    return df_select

In [6]:
df = Tristans_columns(df)

/Users/lourizzolo/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
print(df.shape)
print(df.isnull().sum())
print(df.columns)
print(df.dtypes)

(24953, 6)
policy_id       0
date_start      0
country         0
type            0
ISO_A3        133
compliance    125
dtype: int64
Index(['policy_id', 'date_start', 'country', 'type', 'ISO_A3', 'compliance'], dtype='object')
policy_id              int64
date_start    datetime64[ns]
country               object
type                  object
ISO_A3                object
compliance            object
dtype: object


## Encodage des données compliance

In [23]:
def compliance_mask(df):
    df = df[df['compliance'].notna()]
    mask = df['compliance'].value_counts() > 1000
    df = df[df['compliance'].isin(mask.index)]
    return df

In [24]:
df = compliance_mask(df)

In [25]:
def handle_compliance(x):
    if 'Voluntary/Recommended' in x:
        return 0
    if 'Mandatory (Unspecified/Implied)' in x:
        return 1
    if 'Mandatory with Fines' in x:
        return 2
    if 'Mandatory with Exceptions' in x:
        return 3
    if 'Mandatory with Legal Penalties' in x:
        return 4

In [26]:
def compliance_encode(df):
    df['n_compliance'] = df['compliance'].map(handle_compliance)
    return df
    

In [29]:
df = compliance_encode(df)

In [ ]:
df[df['compliance'].isnull()]

In [ ]:
df = df[df['compliance'].notna()]

In [28]:
print(df.shape)
print(df.isnull().sum())
print(df.columns)
print(df.dtypes)

(24828, 7)
policy_id         0
date_start        0
country           0
type              0
ISO_A3          133
compliance        0
n_compliance    113
dtype: int64
Index(['policy_id', 'date_start', 'country', 'type', 'ISO_A3', 'compliance',
       'n_compliance'],
      dtype='object')
policy_id                int64
date_start      datetime64[ns]
country                 object
type                    object
ISO_A3                  object
compliance              object
n_compliance           float64
dtype: object


In [ ]:
for _ in df['compliance'].unique():
    print('-'*50)
    print(_)

In [ ]:
mask = df['compliance'].value_counts() > 1000

In [ ]:
df = df[df['compliance'].isin(mask.index)]

In [ ]:
df.shape

In [ ]:
df['n_compliance'] = df['compliance'].map(handle_compliance)

In [ ]:
df['n_compliance'].nunique()

In [ ]:
df['ISO_A3'].value_counts()

In [ ]:
df.head()

## Encode type colonne

In [32]:
def get_type(df):
    df = pd.get_dummies(df, columns=["type"])
    return df

In [33]:
get_type(df)

,policy_id,date_start,country,ISO_A3,compliance,n_compliance,type_Anti-Disinformation Measures,type_Closure and Regulation of Schools,type_Curfew,type_Declaration of Emergency,...,type_Internal Border Restrictions,type_Lockdown,"type_New Task Force, Bureau or Administrative Configuration",type_Other Policy Not Listed Above,type_Public Awareness Measures,type_Quarantine,type_Restriction and Regulation of Businesses,type_Restriction and Regulation of Government Services,type_Restrictions of Mass Gatherings,type_Social Distancing
0,2975738,2020-03-06,Afghanistan,AFG,Voluntary/Recommended but No Penalties,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7024226,2020-03-16,Afghanistan,AFG,Mandatory (Unspecified/Implied),1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,889484,2020-05-28,Afghanistan,AFG,Voluntary/Recommended but No Penalties,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,129322,2020-06-28,Afghanistan,AFG,Voluntary/Recommended but No Penalties,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6721435,2020-03-09,Afghanistan,AFG,Mandatory (Unspecified/Implied),1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6018486,2020-03-11,Afghanistan,AFG,Mandatory (Unspecified/Implied),1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,3558044,2020-03-14,Afghanistan,AFG,Mandatory (Unspecified/Implied),1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
18,2231982,2020-04-18,Afghanistan,AFG,Mandatory (Unspecified/Implied),1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
21,1212064,2020-06-03,Afghanistan,AFG,Mandatory (Unspecified/Implied),1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
36,1939072,2020-08-05,Afghanistan,AFG,Voluntary/Recommended but No Penalties,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Missing value for ISO_3

In [34]:
def missing_ISO(df):
    df['ISO_A3'] = np.where(df['ISO_A3'] == '-', df['country'], df['ISO_A3'])
    df['ISO_A3'] = df['ISO_A3'].fillna(df['country'])
    df['ISO_A3'] = df['ISO_A3'].replace('Northern Cyprus', 'CYP')
    df['ISO_A3'] = df['ISO_A3'].replace('Kosovo', 'XK')
    df['ISO_A3'] = df['ISO_A3'].replace('CÙte díIvoire', 'CIV')
    df['ISO_A3'] = df['ISO_A3'].replace('European Union', 'EU')
    df['ISO_A3'] = df['ISO_A3'].replace('S„o TomÈ & PrÌncipe', 'STP')
    df['ISO_A3'] = df['ISO_A3'].replace('Macau', 'MAC')
    df['ISO_A3'] = df['ISO_A3'].replace('Timor Leste', 'TLS')
    return df

In [35]:
missing_ISO(df)

,policy_id,date_start,country,type,ISO_A3,compliance,n_compliance
0,2975738,2020-03-06,Afghanistan,Anti-Disinformation Measures,AFG,Voluntary/Recommended but No Penalties,0.0
1,7024226,2020-03-16,Afghanistan,Anti-Disinformation Measures,AFG,Mandatory (Unspecified/Implied),1.0
2,889484,2020-05-28,Afghanistan,Anti-Disinformation Measures,AFG,Voluntary/Recommended but No Penalties,0.0
3,129322,2020-06-28,Afghanistan,Anti-Disinformation Measures,AFG,Voluntary/Recommended but No Penalties,0.0
4,6721435,2020-03-09,Afghanistan,Closure and Regulation of Schools,AFG,Mandatory (Unspecified/Implied),1.0
7,6018486,2020-03-11,Afghanistan,Closure and Regulation of Schools,AFG,Mandatory (Unspecified/Implied),1.0
10,3558044,2020-03-14,Afghanistan,Closure and Regulation of Schools,AFG,Mandatory (Unspecified/Implied),1.0
18,2231982,2020-04-18,Afghanistan,Closure and Regulation of Schools,AFG,Mandatory (Unspecified/Implied),1.0
21,1212064,2020-06-03,Afghanistan,Closure and Regulation of Schools,AFG,Mandatory (Unspecified/Implied),1.0
36,1939072,2020-08-05,Afghanistan,Closure and Regulation of Schools,AFG,Voluntary/Recommended but No Penalties,0.0


In [ ]:
df['ISO_A3'].nunique()

In [ ]:
df['ISO_A3'] = np.where(df['ISO_A3'] == '-', df['country'], df['ISO_A3'])

In [ ]:
df['ISO_A3'] = df['ISO_A3'].fillna(df['country'])

In [ ]:
df['ISO_A3'].isna().sum()

In [ ]:
df['ISO_A3'] = df['ISO_A3'].replace('Northern Cyprus', 'CYP')
df['ISO_A3'] = df['ISO_A3'].replace('Kosovo', 'XK')
df['ISO_A3'] = df['ISO_A3'].replace('CÙte díIvoire', 'CIV')
df['ISO_A3'] = df['ISO_A3'].replace('European Union', 'EU')
df['ISO_A3'] = df['ISO_A3'].replace('S„o TomÈ & PrÌncipe', 'STP')
df['ISO_A3'] = df['ISO_A3'].replace('Macau', 'MAC')
df['ISO_A3'] = df['ISO_A3'].replace('Timor Leste', 'TLS')

In [ ]:
df['ISO_A3'].unique()

In [ ]:
pd.set_option('display.max_rows', None)
df['ISO_A3'].value_counts()

In [ ]:
df.isna().sum()

 ## Vectorization des données description (pour plus tard)

In [ ]:
def clean(x):
    x = x.lower()
    x = ''.join([letter for letter in x if not letter.isdigit()])
    for p in string.punctuation:
        x = x.replace(p, " ")
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(x)
    tokens = ' '.join([token for token in tokens if token not in stops])
    return tokens

In [ ]:
# données description
df_2['clean_description'] = df_2.description.apply(clean)
df_2['clean_description'] = df_2['clean_description'].astype('str')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df_2.clean_description)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation()

model = lda.fit(X)

In [ ]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        
print_topics(lda, vectorizer)

## Check type_sub_cat

In [ ]:
for _ in df_2['type_sub_cat'].unique():
    print('-'*50)
    print(_)